#Demo de TF-Agents con ambiente Gym para jugar juegos de Atari usando una red DQN

 Basado en los tutoriales: 

   https://www.tensorflow.org/agents/tutorials/2_environments_tutorial 

   https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t#scrollTo=8-AxnvAVyzQQ
   
   https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_04_atari.ipynb 


In [ ]:
#@title Instalar Paquete de TF-Agents
# usar esta versión para evitar errores
# recomendada en https://github.com/tensorflow/agents
!pip install tf-agents[reverb]
!git clone https://github.com/tensorflow/agents.git
!cd agents
#!git checkout v0.15.0  
print("TF-Agentes instalado.")

In [ ]:
#@title Instalar Paquete Gym para acceder a juegos Atari
# nota: hay una version nueva Gymnasium pero todavía no es compatible con TF-Agents
#        https://gymnasium.farama.org/content/gym_compatibility/
!pip install gym[atari,accept-rom-license]     
print("Gym para ATARI instalado.")

In [ ]:
#@title Cargar Librerías

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from random import randint

import random
import pandas as pd

import reverb
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.policies import random_tf_policy
from tf_agents.trajectories import time_step as ts

from tf_agents.agents.dqn import dqn_agent
from tf_agents.agents import CategoricalDqnAgent
from tf_agents.networks import q_network, categorical_q_network
from tf_agents.utils import common

from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory

from tf_agents.networks import sequential
from tf_agents.specs import tensor_spec
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.drivers import py_driver
from tf_agents.policies import py_tf_eager_policy

tf.compat.v1.enable_v2_behavior()

print("Librerías cargadas.")
print("(nota: ignorar el error que tira por diferencias de versiones)")

## Entorno para Juego de Atari

In [ ]:
#@title Preparar funciones auxiliares para visualizar juegos Atari

import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
import os
from IPython import display as ipythondisplay

##from pyvirtualdisplay import Display
##display = Display(visible=0, size=(1400, 900))
##display.start()

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""
def show_env_video(env):
  # trata de encontrar el video que corresponde
  global seleccionaJuego
  encuentraVideo = False
  mp4list = glob.glob('./EnvVideos/*.mp4')
  if len(mp4list) > 0:
    # toma el último video generado para el juego
    mp4list.sort(reverse=True, key=os.path.getmtime)
    mp4 = mp4list[0]
    encuentraVideo = True
  if encuentraVideo:
    print("Video: ", mp4)
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("No se encuentra video " + mp4 + " del juego!")
    
    
def func_episode_trigger(ep):
  # siempre graba
  return True

def wrap_env_recorder(env):
  global seleccionaJuego
  env = RecordVideo(env, 
                    video_folder = './EnvVideos', 
                    episode_trigger = func_episode_trigger, 
                    video_length = 0,
                    name_prefix = seleccionaJuego)
  return env

print("\nWrapper para generar video preparado.")

def simular_entorno(env, policy, mostrarRecompensa=True, num_episodes=1, mostrar_video=True):
  for i in range(num_episodes):
    if num_episodes > 1:
      print("Generando episodio " + str(i+1) + "...")
    else:
      print("Generando...")
    # inicia entorno
    time_step = env.reset()
    sumR = 0.00
    while not time_step.is_last():
      # hace jugar
      action_step = policy.action(time_step)
      time_step = env.step(action_step.action)
      sumR += time_step.reward.numpy()[0]
    # muestra recompensa
    if mostrarRecompensa:
      rFinal = time_step.reward.numpy()[0]
      if num_episodes > 1:
        print("Recompensa Acumulada del episodio " + str(i+1) + ": ", sumR)
        #print("Recompensa Final del episodio " + str(i+1) + ": ", rFinal)
      else:
        print("Recompensa Acumulada: ", sumR)
        #print("Recompensa Final: ", rFinal)
    if mostrar_video:
      show_env_video(env)
  return 

print("\nFunción para simular entorno definida.")

In [ ]:
#@title Seleccionar el juego de Atari

seleccionaJuego = "Breakout" #@param ["Pong", "Freeway", "Enduro", "Asteroids", "Breakout", "Space Invaders"]
#@markdown Ver informacion en https://www.gymlibrary.dev/environments/atari/complete_list/
tipoObsTS = "Grayscale Screen" #@param ["Game RAM", "Grayscale Screen", "RGB Screen"]
entornoDeterministico = True #@param{type:"boolean"}
maxima_cantidad_pasos_juego = 1500 #@param{type:"integer"}

# selecciona juego
if seleccionaJuego == "Freeway":
  gym_env_name = 'ALE/Freeway-v5'
elif seleccionaJuego == "Enduro":
  gym_env_name = 'ALE/Enduro-v5'
elif seleccionaJuego == "Pong":
  gym_env_name = 'ALE/Pong-v5'
elif seleccionaJuego == "Asteroids":
  gym_env_name = 'ALE/Asteroids-v5'
elif seleccionaJuego == "Breakout":
  gym_env_name = 'ALE/Breakout-v5'
elif seleccionaJuego == "Space Invaders":
  gym_env_name = 'ALE/SpaceInvaders-v5'
else:
  raise ValueError("No se puede defnir gym_env_name!!!")

# determina tipo de OBS
if tipoObsTS == "Game RAM":
  obsType = 'ram'
elif tipoObsTS == "RGB Screen":
  obsType = 'rgb'
elif tipoObsTS == "Grayscale Screen":
  obsType = 'grayscale'
else:
  raise ValueError("No se puede defnir obsType!!!")  

# si es negativo le asigna 0 que es cantidad infinita
if maxima_cantidad_pasos_juego < 0:
  maxima_cantidad_pasos_juego = 0

# librerías especiales
from tf_agents.environments import suite_gym


# función para inicializar juego
def inicializar_gym_env(gym_env_name, obs_type, entornoDeterministico=False, max_episode_steps=1000, grabaSteps=True):
    if grabaSteps:
      gym_env_wrappers = [wrap_env_recorder]
    else:
      gym_env_wrappers = None

    if entornoDeterministico:
      # crea el entorno con parámetros deterministicos
      env =suite_gym.load(gym_env_name, 
                                gym_env_wrappers = gym_env_wrappers,
                                max_episode_steps=max_episode_steps, 
                                gym_kwargs={'frameskip': 1,
                                            'repeat_action_probability':False,
                                            'full_action_space':False,
                                            'obs_type':obsType,
                                            'render_mode':'rgb_array'}
                          )
    else:
       # crea el entorno con parámetros estocásticos
      env =suite_gym.load(gym_env_name, 
                                gym_env_wrappers = gym_env_wrappers,
                                max_episode_steps=max_episode_steps, 
                                gym_kwargs={'full_action_space':False,
                                            'obs_type':obsType,
                                            'render_mode':'rgb_array'}
                          )

    env.metadata['render_fps'] = 30
    return env


# crea el entorno
atari_py_env = inicializar_gym_env(
                  gym_env_name, 
                  obs_type = obsType, 
                  entornoDeterministico = entornoDeterministico, 
                  max_episode_steps = maxima_cantidad_pasos_juego, 
                  grabaSteps = True)

# Definir wrapper para convertir en entornos TF
atari_env = tf_py_environment.TFPyEnvironment(atari_py_env)

# asigna nombre variables por compatibilidad código para entrenar
train_py_env = atari_py_env
eval_py_env = atari_py_env
train_env = atari_env
eval_env = atari_env

# define política al azar independiente del Agente
random_policy = random_tf_policy.RandomTFPolicy(atari_env.time_step_spec(), 
                                                atari_env.action_spec())
# muesta información del entorno
print("\n")
print('- Entorno: ', gym_env_name)
print('\n- Specification:')
for det in atari_py_env._env._gym_env.spec.kwargs:
  print("  ", det, "=", atari_py_env._env._gym_env.spec.kwargs[det])
#print("   max_episode_steps=", atari_py_env._env._gym_env._max_episode_steps)
print('\n- Time Step Spec:')
print("  ", atari_env.time_step_spec())
print('\n-Action Spec:')
print("  ", atari_env.action_spec())
print('\n-Reward range:')
print("  ", atari_py_env._env._gym_env.reward_range)

# muestra pantalla ejemplo
print("\n-Ejemplo pantalla: ")
atari_env.reset()
import PIL.Image
PIL.Image.fromarray(atari_py_env.render())

In [ ]:
#@title Ejemplo de juego jugando al Azar
simular_entorno(atari_env, random_policy, True, 1, True)


##DQN

In [ ]:
#@title Definir el Agente tipo DQN
entrenar_DQN = True # @param {type:"boolean"}
DQNpolicy = None

if entrenar_DQN:
  # agente categorical oculto por ahora (problemas en librería)
  tipo_agente = "DQN" #param ["DQN", "DQN Categorico (C51)"]
  learning_rate = 1e-3  # @param {type:"number"}
  cant_neuronas_ocultas = "100, 50, 25" # @param {type:"string"}
  DQNCat_num_atoms = 51  #param {type:"integer"}

  # controla cantidad de atoms para DQN Cat
  if DQNCat_num_atoms <= 1:
    DQNCat_num_atoms = 51

  #define las capas convolutional

  #define las capas convolutional
  if obsType == 'rgb':
    # como es una matriz/imagen se usa CNN
    CNN_preprocessing_layers = tf.keras.models.Sequential(
                                        [tf.keras.layers.LayerNormalization(axis=1),                                       
                                        tf.keras.layers.Conv2D(2, 2, activation='relu', padding="same"),
                                        tf.keras.layers.MaxPooling2D(pool_size=(2)),
                                        tf.keras.layers.Conv2D(2, 2, activation='relu', padding="same"),
                                        tf.keras.layers.MaxPooling2D(pool_size=(2)),
                                        tf.keras.layers.Flatten()])
    print("Agrega capas CNN para preprocesamiento de entrada RGB.")
  elif obsType == 'grayscale':
    # como es una matriz/imagen se usa CNN
    CNN_preprocessing_layers = tf.keras.models.Sequential(
                                        [tf.keras.layers.LayerNormalization(axis=1),                                        
                                        tf.keras.layers.Conv1D(2, 2, activation='relu', padding="same"),
                                        tf.keras.layers.MaxPooling1D(pool_size=(2)),
                                        tf.keras.layers.Flatten()])
    print("Agrega capas CNN para preprocesamiento de entrada grayscale.")
  else:
    # como es un vector no se usa CNN
    CNN_preprocessing_layers = None

  # Define cantidad de neuronas ocultas para RNA-Q
  hidden_layers = []
  for val in cant_neuronas_ocultas.split(','):
    if  int(val) < 1:
      hidden_layers.append( 10 )
    else:
      hidden_layers.append( int(val) )
  fc_layer_params = tuple(hidden_layers, )
  #print(fc_layer_params)

  if tipo_agente=="DQN":

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    train_step_counter = tf.Variable(0)

    # Define RNA-Q
    q_net = q_network.QNetwork(
        train_env.observation_spec(),
        train_env.action_spec(),
        preprocessing_layers=CNN_preprocessing_layers,
        fc_layer_params=fc_layer_params)

    # Define el agente de tipo Q
    ag = dqn_agent.DqnAgent(
        train_env.time_step_spec(),
        train_env.action_spec(),
        q_network=q_net,
        optimizer=optimizer,
        td_errors_loss_fn=common.element_wise_squared_loss,
        train_step_counter=train_step_counter)

    ag.initialize()

    print("Agente DQN inicializado. ")

  elif tipo_agente == "DQN Categorico (C51)":
    
    # Define RNA-Q Categórico
    categorical_q_net = categorical_q_network.CategoricalQNetwork(
        train_env.observation_spec(),
        train_env.action_spec(),
        num_atoms=DQNCat_num_atoms,
        preprocessing_layers=CNN_preprocessing_layers,
        fc_layer_params=fc_layer_params)

    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

    train_step_counter = tf.compat.v2.Variable(0)
    
    # parámetros especificos (por defecto)
    n_step_update = 2
    gamma = 0.99

    # Define el agente de tipo Q Categórico
    ag = CategoricalDqnAgent(
        train_env.time_step_spec(),
        train_env.action_spec(),
        categorical_q_network=categorical_q_net,
        optimizer=optimizer,
        n_step_update=n_step_update,
        td_errors_loss_fn=common.element_wise_squared_loss,
        gamma=gamma,
        train_step_counter=train_step_counter)
    
    ag.initialize()
    
    print("Agente DQN Categorico (C51) inicializado. ")
else:
  print("No se ejecuta entrenamiento de Agente DQN.")  

In [ ]:
#@title Métricas para evaluación y Preparar datos para Entrenamiento del Agente DQN

if entrenar_DQN:

  # parámetros
  initial_collect_steps = 500  # @param {type:"integer"} 
  collect_steps_per_iteration = 10# @param {type:"integer"}
  replay_buffer_max_length = 100000  # @param {type:"integer"}
  batch_size = 64  # @param {type:"integer"}
  num_eval_episodes = 10  # @param {type:"integer"}

  # Definir Métricas para evaluación para Agente DQN
    
  #eval_policy = ag.policy
  #collect_policy = ag.collect_policy
  #time_step = train_env.reset()

  # Se usa el promedio de la recompensa (la más común)
  # See also the metrics module for standard implementations of different metrics.
  # https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

  def compute_avg_return(environment, policy, num_episodes=10):
    if num_episodes == 0:
        return 0.0 
    total_return = 0.0
    for _ in range(num_episodes):

      time_step = environment.reset()
      episode_return = 0.0

      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = environment.step(action_step.action)
        episode_return += time_step.reward
      total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

  ##compute_avg_return(eval_env, random_policy, num_eval_episodes)

  # Define 'Replay Buffer' para que el agente recuerde las observaciones realizadas

  table_name = 'uniform_table'
  replay_buffer_signature = tensor_spec.from_spec(
        ag.collect_data_spec)
  replay_buffer_signature = tensor_spec.add_outer_dim(
      replay_buffer_signature)

  table = reverb.Table(
      table_name,
      max_size=replay_buffer_max_length,
      sampler=reverb.selectors.Uniform(),
      remover=reverb.selectors.Fifo(),
      rate_limiter=reverb.rate_limiters.MinSize(1),
      signature=replay_buffer_signature)

  reverb_server = reverb.Server([table])

  replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
      ag.collect_data_spec,
      table_name=table_name,
      sequence_length=2,
      local_server=reverb_server)

  rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
    replay_buffer.py_client,
    table_name,
    sequence_length=2)      

  print("\nDatos recolectados.")

  # Dataset generates trajectories with shape [Bx2x...]
  dataset = replay_buffer.as_dataset(
      num_parallel_calls=3,
      sample_batch_size=batch_size,
      num_steps=2).prefetch(3)
  iterator = iter(dataset)
  
  print("\nDataset creado para datos recolectadso.")

else:
  print("No se ejecuta entrenamiento de Agente DQN.")  

In [ ]:
#@title Entrenar al Agente DQN

if entrenar_DQN:

  cant_ciclos_entrenamiento_finalizar = 20000# @param {type:"integer"}
  log_cada_ciclos = 200  # @param {type:"integer"}
  mostar_recompensa_cada = 500  # @param {type:"integer"}
  cant_episodios_evaluacion =  10# @param {type:"integer"}
  minima_recompensa_promedio_finalizar = 10.0 # @param {type:"number"}
  
  # (Optional) Optimize by wrapping some of the code in a graph using TF function.
  ag.train = common.function(ag.train)

  # Reset the train step.
  ag.train_step_counter.assign(0)

  # Evaluate the agent's policy once before training.  
  avg_return = compute_avg_return(eval_env, ag.policy, cant_episodios_evaluacion)
  ar_cicloL = []
  ar_cicloR = []
  ar_returns = []
  ar_loss = []

  # Reset the environment.
  time_step = train_py_env.reset()

  # Create a driver to collect experience.
  collect_driver = py_driver.PyDriver(
      train_py_env,
      py_tf_eager_policy.PyTFEagerPolicy(
        ag.collect_policy, use_tf_function=True),
      [rb_observer],
      max_steps=collect_steps_per_iteration)

  print("\n** Comienza el Entrenamiento **\n")
  for _ in range(cant_ciclos_entrenamiento_finalizar):

    # Collect a few steps and save to the replay buffer.
    time_step, _ = collect_driver.run(time_step)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    try:
      train_loss = ag.train(experience).loss
    except:
      # valor para error 
      train_loss = -999

    step = ag.train_step_counter.numpy()    

    if (step == 1) or (step == cant_ciclos_entrenamiento_finalizar) or (step % log_cada_ciclos == 0):
      if train_loss == -999:
        print('step = {0}: ERROR al calcular LOSS!'.format(step))      
      else:
        print('step = {0}: loss = {1:.3f}'.format(step, train_loss))    
        ar_cicloL.append( step )
        ar_loss.append( train_loss )
    
    if (step == 1) or (step == cant_ciclos_entrenamiento_finalizar) or (step % mostar_recompensa_cada == 0):
      avg_return = compute_avg_return(eval_env, ag.policy, cant_episodios_evaluacion)
      ar_cicloR.append( step )
      ar_returns.append( avg_return )
      print('step = {0}: Promedio Recompensa = {1:.1f}'.format(step, avg_return))

      if (avg_return >= minima_recompensa_promedio_finalizar):
        print('** Finaliza en step {0} por buen valor de recompensa promedio: {1:.1f}'.format(step, avg_return)) 
        break

  DQNpolicy = ag.policy
  print("\n** Entrenamiento Finalizado **\n")
else:
  print("No se ejecuta entrenamiento de Agente DQN.")  

In [ ]:
#@title Mostrar Gráficos del Entrenamiento del Agente DQN
if entrenar_DQN:

  plt.figure(figsize=(12,5)) 
  plt.plot( ar_cicloR, ar_returns)
  plt.title("Resultados del Entrenamiento del Agente - Promedio Recompensa")
  #plt.legend(['Promedio Recompensa', 'Loss de Entrenamiento'], loc='upper right')
  plt.ylabel('Valor')
  plt.xlabel('Ciclo')
  plt.xlim(right=max(ar_cicloR))   
  plt.grid(True)
  plt.show()

  plt.figure(figsize=(12,5)) 
  plt.plot( ar_cicloL, ar_loss, color="red" )
  plt.title("Resultados del Entrenamiento del Agente - Loss de Entrenamiento")
  #plt.legend(['Promedio Recompensa', 'Loss de Entrenamiento'], loc='upper right')
  plt.ylabel('Valor')
  plt.xlabel('Ciclo')
  plt.xlim(right=max(ar_cicloL))   
  plt.grid(True)
  plt.show()


In [ ]:
#@title Probar el Agente DQN Entrenado 
simular_entorno(atari_env, DQNpolicy, True, 1, True)


In [ ]:
#@title Cargar o Guardar el Agente DQN entrenado

# parámetros
directorio_modelo = '/content/gdrive/MyDrive/IA/demoRL/Modelos' #@param {type:"string"}
nombre_modelo_grabar = "policy-Atari" #@param {type:"string"}
accion_realizar = "-" #@param ["-", "Cargar Modelo", "Grabar Modelo"]

if accion_realizar != "-":
  import os
  from google.colab import drive
  from tf_agents.policies import TFPolicy, policy_saver
  # determina lugar donde se guarda el modelo
  policy_dir = os.path.join(directorio_modelo, nombre_modelo_grabar)
  policy_dir = os.path.join(policy_dir, gym_env_name)
  # Montar Drive
  drive.mount('/content/gdrive')
if accion_realizar == "Grabar Modelo":
  if (DQNpolicy is not None) and isinstance(DQNpolicy, TFPolicy):
    # guarda la politica del agente DQN entrenado
    tf_policy_saver = policy_saver.PolicySaver(DQNpolicy)
    tf_policy_saver.save(policy_dir)
    print("\nPolítica DQN guardada en ", policy_dir)
elif accion_realizar == "Cargar Modelo":
  # carga la política del modelo
  DQNpolicy = tf.compat.v2.saved_model.load(policy_dir)
  print("\nPolítica DQN recuperada de ", policy_dir)
